# Теория вероятностей и математическая статистика

## Домашнее задание №1

*Резепова Софья*

### Задача 1 (5 баллов)

В CSV-файле в папке [problem01](https://disk.yandex.ru/d/PxeNlHpen-eJ0A) хранятся результаты AB-тестирования сайта продуктового магазина. Менеджеры по продажам заинтересованы в том, чтобы посетители сайта скачивали приложение магазина и становились участниками программы лояльности, поэтому ключевой вопрос звучит так: нужно ли изменять внешний вид ссылки на скачивание приложения, чтобы привлечь больше покупателей к участию в программе лояльности, или нет?

В текущей версии сайта пользователям предлагается поле для ввода номера телефона для отправки ссылки на приложение в SMS и опция **Show link** для просмотра самой ссылки. В новой версии сайта пользователям предлагается стандартное окно **Our Mobile Apps** с кнопками-ссылками на App Store и Google Play.

В течение некоторого времени посетителям сайта случайным образом показывали текущую версию сайта (с серверов под номерами 2 и 3) и обновлённую версию сайта (с сервера под номером 1), а потом фиксировали, перешли ли пользователи на страницу для скачивания приложения или нет. 

Показатели в файле:

* `RecordID`: id записи;
* `IP.Address`: IP адрес посетителя сайта;
* `LoggedInFlag`: зашёл ли пользователь в аккаунт на сайте (0 – нет, 1 – да);
* `ServerID`: id сервера, с которого открывается сайт;
* `VisitPageFlag`: перешёл ли пользователь на страницу для скачивания приложения и регистрации в программе лояльности.

**Конверсия** в данном случае – процент пользователей, которые перешли на страницу для скачивания приложения.

Загрузите данные из файла и оставьте в датафрейме только те строки, которые соответствуют незарегистрированным пользователям или пользователям, не зашедшим в аккаунт (столбец `LoggedInFlag`).

Используя предложенные данные, проверьте на 5% гипотезу о равенстве конверсий старой и новой версии сайта против односторонней (направленной) альтернативы. Направление выберите, исходя из данных. Можно ли считать, что новый дизайн сайта потенциально увеличивает конверсию?

Для проверки гипотезы используйте функцию [proportions_ztest()](https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportions_ztest.html).

Ваше решение должно содержать:

* *(1 балл)* Сформулированные нулевую и альтернативную гипотезы.
* *(3 балла)* Код Python для проверки гипотезы. 
* *(1 балл)* Вывод относительно нулевой гипотезы (отвергается/не отвергается, почему) и ответ на вопрос об увеличении/не увеличении конверсии.

H0: p1 = p2 (конверсия старого сайта и нового сайта равны)
\
H1: p2 > p1 (конверсия новой версии сайта больше, чем старой)

In [ ]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv('Резепова Софья Владимировна1.csv')

df = df[df['LoggedInFlag'] == 0]

old = df[df['ServerID'].isin([2, 3])]
new = df[df['ServerID'] == 1]


old_true = old['VisitPageFlag'].sum()
new_true = new['VisitPageFlag'].sum()

count = [new_true, len(new)]
nobs = [old_true, len(old)]

z_stat, p_value = proportions_ztest(count, nobs, alternative='larger') #порядок важен, проверяем лучше ли новая версия, значит новая версия первая

print(f'{z_stat:.5f}')
print(f'{p_value:.5f}')


5.31015
0.00000


p - value < 0,05 ==> нулевая гипотеза отвергается на уровне значимости 5%, новая версия сайта повышает конверсию

### Задача 2 (10 баллов)

В CSV-файле в папке [problem02](https://disk.yandex.ru/d/yAtFV351Xc-tLg) сохранены результаты обследования участников эксперимента, которые следовали разным типам диеты. Переменные в файле:

* `Gender`: пол участника (0 – женский, 1 – мужской);
* `Age`: возраст участника;
* `Height`: рост участника (в см);
* `Diet`: тип диеты (`A`, `B`, `C`);
* `Diff`: разница в весе до диеты и после, количество сброшенных килограммов после 6 недель диеты;
* `WeightS`: вес участника на начало эксперимента (в кг).

Необходимо сравнить эффективность диет A и B, считая, что эффективность диеты измеряется через среднее значение количества сброшенных килограммов. Предполагаем, что этот показатель имеет нормальное распределение.

**Часть 1**. Для выбора варианта критерия Стьюдента для двух независимых выборок, необходимо проверить гипотезу о равенстве дисперсий. Проверьте её на 10\%-ном уровне значимости против направленной альтернативы.

Для этого реализуйте F-критерий, вычислив в Python наблюдаемое значение статистики критерия и p-value. Для вычисления p-value воспользуйтесь функцией [f()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f.html) из модуля `stats` библиотеки `scipy` для создания случайной величины с подходящими параметрами распределения и соответствующими методами для вычисления вероятностей.

Ваше решение должно содержать:

* *(1 балл)* Сформулированные нулевую и альтернативную гипотезы.
* *(3 балла)* Код Python для проверки гипотезы. 
* *(1 балл)* Вывод относительно нулевой гипотезы (отвергается/не отвергается, почему).

**Часть 2**. Проверьте гипотезу о равенстве среднего количества сброшенных килограммов при следовании диете A и диете B против двусторонней (ненаправленной) альтернативы, приняв уровень значимости равным 5%.

Для этого воспользуйтесь функцией [ttest_ind()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) из модуля `stats` библиотеки `scipy` и выберите подходящий вариант теста с учётом выводов из первой части задачи.

Ваше решение должно содержать:

* *(1 балл)* Сформулированные нулевую и альтернативную гипотезы.
* *(3 балла)* Код Python для проверки гипотезы. 
* *(1 балл)* Вывод относительно нулевой гипотезы (отвергается/не отвергается, почему) и заключение, можно ли считать одну диету более эффективной, чем другая, в предположении, что все необходимые условия для возможности таких выводов в эксперименте соблюдены.

H0: дисперсия количества сброшенных кг при диете А равна дисперсии количества сброшенных кг при диете В
\
Н1: дисперсия при диете А больше дисперсии при диете В

In [ ]:
import pandas as pd
df = pd.read_csv('Резепова Софья Владимировна.csv')

df_a = df[df['Diet'] == 'A']['Diff']
df_b = df[df['Diet'] == 'B']['Diff']

from scipy import stats

var_a = df_a.var(ddof=1)
var_b = df_b.var(ddof=1)

if var_a > var_b:
    f_stat = var_a / var_b
else:
    f_stat = var_b / var_a

df1 = len(df_a) - 1
df2 = len(df_b) - 1

p_value = 1 - stats.f.cdf(f_stat, df1, df2) #правостороняя вероятность: площадь справа (так как проверяем дисперсия А БОЛЬШЕ дисперсии В)

print(f"F-статистика: {f_stat}")
print(f"p-value: {p_value}")


F-статистика: 1.1815831008555546
p-value: 0.3146823210519176


p - value > 0,1 ==> нет оснований отвергать нулевую гипотезу; дисперсии можно считать равными

Н0: среднее количество сброшенных кг одинаково для диет А и В (μ_A = μ_B)
\
Н1: среднее количество сброшенных кг не одинаково для диет А и В (μ_A ≠ μ_B)

In [33]:
t_stat, p_val = stats.ttest_ind(df_a, df_b)

print(f"t-статистика: {t_stat}")
print(f"p-value: {p_val}")


t-статистика: -0.6335615478623933
p-value: 0.5284922980720586


p-value > 0.05 ==> нет оснований отвергать нулевую гипотезу; нельзя считать одну диету эффективнее другой

### Задача 3 (5 баллов)

Используя данные из предыдущей задачи для диеты A, покажите, что сравнение средних в случае парных (связанных) выборок эквивалентно сравнению среднего значения разности средних этих выборок с нулём.

Другими словами, для диеты A сравните результаты:

* двухвыборочного критерия Стьюдента для связанных выборок – *related samples*, отсюда функция [ttest_rel()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html#scipy.stats.ttest_rel).
* одновыборочного критерия Стьюдента – функция [ttest_1samp()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html#scipy.stats.ttest_1samp).

Какие выборки считать связанными и что сравнивать с 0 – определите самостоятельно, опираясь на постановку задачи, дизайн эксперимента и столбцы в датафрейме.

In [ ]:
w0 = df_a['WeightS']
w1 = df_a['WeightS'] - df_a['Diff']

t_rel, p_rel = stats.ttest_rel(w0, w1)

t_1samp, p_1samp = stats.ttest_1samp(df_a['Diff'], popmean=0)

print(f"ttest_rel: t = {t_rel}, p = {p_rel}")
print(f"ttest_1samp: t = {t_1samp}, p = {p_1samp}")


ttest_rel: t = 8.239850245835642, p = 1.2937275504677139e-09
ttest_1samp: t = 8.239850245835642, p = 1.2937275504677139e-09


t-тест для связанных выборок и одновыборочный t-тест дают одинаковый результат, так как математически проверяют одну и ту же гипотезу

### Задача 4 (5 баллов)

В файле с расширением `.npy` (формат, в который можно выгрузить массив NumPy) в папке [problem03](https://disk.yandex.ru/d/KKpXssdt0qaZLw) сохранены два массива с выборками. Ниже приведён пример кода для загрузки массивов из такого файла:

In [17]:
import numpy as np

# название/путь к файлу
data = np.load("Резепова Софья Владимировна.npy")
sample01, sample02 = data

Проверьте для каждой выборки на 5%-ном уровне значимости гипотезу о том, что выборка взята из нормального распределения. Если обе выборки взяты из нормального распределения, реализуйте подходящий вариант критерия Стьюдента для сравнения средних, если нет – реализуйте критерий Манна-Уитни для сравнения распределений. Уровень значимости тот же.

Функции для реализации тестов – см. [здесь](https://docs.scipy.org/doc/scipy/reference/stats.html) в разделе *Independent Sample Tests*. Для проверки нормальности можете использовать [критерий](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html#scipy.stats.shapiro) Шапиро-Уилка.


Ваше решение должно содержать:

* *(2 балла)* Код Python для проверки гипотезы о нормальности распределения, выводы, можно ли считать распределения, из которого взяты выборки, нормальным.
* *(1 балл)* Сформулированные нулевую и альтернативную гипотезы о средних/распределениях (альтернативу считайте двусторонней).
* *(2 балла)* Код Python для проверки нулевой гипотезы и вывод относительно нулевой гипотезы (отвергается/не отвергается, почему).

In [20]:
from scipy.stats import shapiro, ttest_ind, mannwhitneyu

In [21]:
shapiro_1 = shapiro(sample01)
shapiro_2 = shapiro(sample02)

print(f"p-value первой выборки = {shapiro_1.pvalue:.4f}")
print(f"p-value второй выборки = {shapiro_2.pvalue:.4f}")

p-value первой выборки = 0.0000
p-value второй выборки = 0.5783


p-value первой выборки < 0.05 ==> распредение не нормальное
\
p-value второй выборки > 0.05 ==> распредение нормальное

одна из выборок не нормальна ==> используем критерий Манна-Уитни

H0: распределения одинаковы
\
Н1: распредения различаются

In [23]:
mw = mannwhitneyu(sample01, sample02, alternative='two-sided')
print(f"p-value: {mw.pvalue:.4f}")

p-value: 0.0000


p-value < 0.05 ==> нулевая гипотеза отвергается, распределения различаются